Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [229]:
from random import choices,randint

import lab9_lib

In [230]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

00110110111010011111010110111100111000101011000011: 9.11%
00000011100100101101101100100000000101110111111100: 15.56%
11001101111101000011011111000001110100001011100111: 15.33%
01101010001010111111011101101101010111110110000111: 39.34%
10100101001001000010011010010111110100111100010101: 7.33%
11001110000100101110111011010011000111001100100000: 9.11%
11111111011000010011011001100111010011000011011011: 23.33%
00000011000001011000001000001001000100000101100100: 19.78%
11101100111101000101011101000110111010010001001000: 15.33%
00110111010001110010011101011111010101011010011100: 9.11%
10


## EA

In [231]:
import random


def mutation(genome):
    index = randint(0,len(genome[0])-1)
    genome[0][index] = 1-genome[0][index]
    return genome[0]


def one_cut_xover(ind1, ind2):
    cut_point = randint(0, len(ind1[0]))
    offspring = ind1[0][:cut_point]+ind2[0][cut_point:]
    return offspring

def xover(genome1, genome2):
    child_genome = [g1 if random.random() > 0.5 else g2 for g1, g2 in zip(genome1[0], genome2[0])]
    return child_genome

In [232]:
from random import choice


def select_parent(population,fitness):
    best_parents = sorted(population,key= lambda i:i[1],reverse=True)[:int(len(population)/2)]
    pool = [choice(best_parents) for _ in range(int(len(population)/4))]
    champion = max(pool, key=lambda i: i[1])
    return champion

In [233]:
def init_population(n_individual,length,fitness):
    pop = []
    for _ in range(n_individual):
        ind = (choices([0, 1], k=length))
        pop.append((ind,fitness(ind)))
    return pop

In [234]:
def gen_new_population(n_new_indivdual,mutation_prob,old_population,fitness):
    new_individual = []
    for _ in range(n_new_indivdual):
        if random.random() < mutation_prob:
            old_ind = select_parent(old_population,fitness)
            tmp = mutation(old_ind)
        else:
            old_ind = select_parent(old_population,fitness)
            old_ind_2 = select_parent(old_population,fitness)
            tmp = one_cut_xover(old_ind,old_ind_2)
        new_individual.append((tmp,fitness(tmp)))
    return new_individual

In [235]:
def replacement(new_pop,old_pop,fitness):
    tmp_pop = new_pop + old_pop
    sorted_pop = sorted(tmp_pop,key= lambda i:i[1],reverse=True)
    return sorted_pop[:len(new_pop)]

In [236]:
N_INDV = 500
LENGTH_INDV = 1000
GENERATION = 300
fit = lab9_lib.make_problem(10)
pop = init_population(N_INDV,LENGTH_INDV,fit)

for g in range(GENERATION):
    if g%10 == 0:
        avg_fit = sum(list(map(lambda i:i[1],pop)))/len(pop)
        print(pop[0][1], avg_fit)
    new_pop = gen_new_population(N_INDV,0.1,pop,fit)
    pop = replacement(new_pop,pop,fit)
    
print(fit.calls)

0.050811123464 0.05907650716564001
0.227001127 0.2268960746060002
0.23502224000000002 0.2312449624659997
0.235356783 0.23518443745599948
0.235777904 0.23567284270399821
0.236011127 0.23594355666799935
0.236334449 0.23623615383800026
0.236666671 0.2365224058980003
0.240788904 0.24040873332000037
0.24111122499999998 0.24099762585599965
0.241233458 0.24113435586800044
0.24534455900000002 0.2452581770680003
0.245677781 0.24558813023400014
0.245889003 0.24582860588400035
0.246111124 0.24611111420600118
0.250123346 0.24855118295599957
0.250344558 0.25025088102399917
0.250556668 0.2505031825200013
0.25067779 0.2506706805840004
0.250789001 0.25077857231799955
0.250900112 0.2508870768260018
0.251000225 0.25100009356399905
0.25501223500000003 0.2525755812820002
0.259222234 0.25912057703200087
0.259233445 0.2592246056179982
0.259333445 0.25923464310000144
0.263134457 0.25927364395799857
0.263235567 0.2632354135500014
0.26345556800000003 0.26341017815400064
0.267555578 0.2675543719180024
150500
